Install playwright into the Colab environment

In [ ]:
!pip install playwright
!playwright install


Import necessary libraries

In [ ]:
import asyncio
from playwright.async_api import async_playwright
from google.colab import drive
from google.colab import auth
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload
from openpyxl import load_workbook, Workbook
from googleapiclient.discovery import build
from oauth2client.client import GoogleCredentials
import io
import re

Setup connectivity to google drive

In [ ]:
# Mount Google Drive to access folders directly
print("Mounting Google Drive...")
drive.mount('/content/drive')

# Authenticate for Google API access
auth.authenticate_user()


# Function to get credentials and create Drive service
def create_drive_service():
    creds = GoogleCredentials.get_application_default()
    return build('drive', 'v3', credentials=creds)

# Set the target Google Drive folder ID (from your provided URL)
drive_folder_id = "1MnCNbtItDKzOJPJQcvJqJvAsIOfwNqGx"
print(f"Target folder ID: {drive_folder_id}")


# Upload function to Google Drive
def upload_to_drive(file_path, file_name):
    # Initialize Google Drive service
    drive_service = create_drive_service()

    file_metadata = {'name': file_name, 'parents': [drive_folder_id]}
    media = MediaFileUpload(file_path, mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')

    # Create request to upload the file
    request = drive_service.files().create(media_body=media, body=file_metadata)
    response = request.execute()
    print(f"Uploaded: {response['name']} to Google Drive")




Create a function that downloads all of the excel files from the website.  This is the scraping function

In [ ]:
async def main():
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto("https://www.ed.gov/about/ed-overview/budget/estimated-esea-title-i-lea-allocations-fy-2024")

        links = await page.query_selector_all("a")

        for link in links:
            href = await link.get_attribute("href")
            if href and href.endswith(".xlsx"):
                async with page.expect_download(timeout=60000) as download_info:
                    await link.click()
                download = await download_info.value
                file_path = await download.path()
                file_name = download.suggested_filename

                # Upload to Google Drive
                upload_to_drive(file_path, file_name)

        await browser.close()

await main()



Clear all of the sheets.  This is to ensure that files from previous tests are cleared.  This is not necessary unless cleaning the folder for organization.

In [ ]:
# Remove all sheets except the first one in the master Google Sheet
#def clear_extra_sheets(spreadsheet_id):
    sheets_service = build('sheets', 'v4', credentials=GoogleCredentials.get_application_default())
    sheet_metadata = sheets_service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
    sheets = sheet_metadata.get('sheets', [])

    # Keep the first sheet, delete the rest
    for sheet in sheets[1:]:
        sheet_id = sheet['properties']['sheetId']
        title = sheet['properties']['title']
        try:
            sheets_service.spreadsheets().batchUpdate(
                spreadsheetId=spreadsheet_id,
                body={
                    'requests': [{
                        'deleteSheet': {'sheetId': sheet_id}
                    }]
                }
            ).execute()
            print(f"Deleted sheet: {title}")
        except Exception as e:
            print(f"Failed to delete sheet {title}: {e}")

# ID
master_sheet_id = '1aQjTdy3WbWBGHwCO8_UDT0zoGhejCzfmHnyZ6g3qV8k'

# Call this before adding new tabs
#clear_extra_sheets(master_sheet_id)

Add all of the files into one master file separated by tabs

In [ ]:
drive_service = create_drive_service()
sheets_service = build('sheets', 'v4', credentials=GoogleCredentials.get_application_default())

# Get Excel files in folder
results = drive_service.files().list(
    q=f"'{drive_folder_id}' in parents and mimeType='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'",
    fields="files(id, name)"
).execute()

excel_files = results.get('files', [])

for file in excel_files:
    file_id = file['id']
    file_name = file['name']

    # Use regex to extract the state name
    match = re.search(r'tables-([a-z\-]+)-\d+\.xlsx', file_name)
    sheet_name = match.group(1).replace("-", " ").title()[:31] if match else file_name.split('.')[0][:31]

    # Download the Excel file
    request = drive_service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while not done:
        _, done = downloader.next_chunk()
    fh.seek(0)

    try:
        wb = load_workbook(fh)
        ws = wb.active

        # Extract data from Excel
        values = [[cell.value for cell in row] for row in ws.iter_rows()]

        # Create new sheet in Google Sheet
        sheets_service.spreadsheets().batchUpdate(
            spreadsheetId=master_sheet_id,
            body={
                'requests': [{
                    'addSheet': {
                        'properties': {'title': sheet_name}
                    }
                }]
            }
        ).execute()

        # Push data to the new sheet
        sheets_service.spreadsheets().values().update(
            spreadsheetId=master_sheet_id,
            range=f"{sheet_name}!A1",
            valueInputOption="RAW",
            body={"values": values}
        ).execute()

        print(f"Added {file_name} as tab: {sheet_name}")
    except Exception as e:
        break
